### Convolution Neural Networks

These are state of the art Neural Networks that are predominantly used for Images,etc.

When we normally treat an image we consider each of it's pixels and apply algorithms. But when we use CNN, we use relation pf pixels with each other as well. Like if adjoiing pixels represnt a particular objext etc.

#### How it Works?

We apply a filter on the Image and create a new image of sorts.

Let us assume we have a 20 x 20 sized image. and we run a 3 x 3 filter across it. That means we keep the 3 x 3 filter across the first pixel of the image, then the next and so on. 

We move the filter across all the pixels of the image. When we apply the filter on a segment of the image we do multiply each value of the filter with the corresponding value of pixel and we generate an output value like that

#### Stride and Padding

Stride is deciding by how much the filter has to be moved. Usually it is 1 but we can change this.

Padding - zero padding or no padding is done to give corner pixels equal weightage as other pixels as corner pixels are used only once per filter unlike internal pixels

#### What to do in case of Coloured Images?

Coloured Images are 3-D Images of RGB.

In order to comb through these 3-D Images we have to use a 3-D filter as well.

For Example we have a 28 x 28 x 3 Image. We will need a 4 x 4 x 3 Filter to apply on it. And then we will have to have  of those for  differnet directions etc

#### Pooling Layer

A pooling layer is applied generally after every convolution layer. It helps to reduce the sizeof O/P and helps us train our NN faster.

#### Code

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

In [ ]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [ ]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels,n_conv1])),
    'wc2' : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    'wh1' : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    'wo' : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([n_conv1])),
    'bc2' : tf.Variable(tf.random_normal([n_conv2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out])) 
}

In [ ]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding = "SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k=2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1,k,k,1], strides = [1,k,k,1])

In [ ]:
def cnn(x,weights,biases,keep_prob):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'])
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['b2'])
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [ ]:
x = tf.placeholder("float",[None, input_pixels])
y = tf.placeholder(tf.int32,[None, n_out])
keep_prob = tf.placeholder["float"]
pred = cnn(x,weights,biases,keep_prob) 

In [ ]:
cost = tf.reduce_mean(f.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels =y))

In [ ]:
optimizer = tf.train.AdamOptimzer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
batch-size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c,d = sess.run([cost,optimize], feed_dict = {x:batch_x,y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

In [ ]:
predictions = tf.argmax(pred,1)
true_labels = tf.argmax(y,1)
correct_predictions = tf.equal(predictions,true_labels)
predictions_eval,labels,correct_pred = sess.run([predictions,true_labels,correct_predictions],feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})
predictions_eval,labels,correct_pred
correct_pred.sum()

#### Dropout Layer

Dropout Layer is added to avoid overfitting.

It's a layer which is added in between hidden layers which gets rid of certain probabilities to avoid Overfitting.